# Building a Research Agent using Agno

This notebook demonstrates how to build and use a **Research Agent** leveraging the [Agno framework](https://github.com/agno-agi/agno) alongside tools like [Tavily](https://github.com/tavily-ai/tavily-python) for web search and the [Contextual AI](https://contextual.ai/) for RAG search.  

The workflow is organized into three major sections, each designed to progressively introduce you to agent construction, reasoning, and advanced tool integration.

---

## 1. Setup the Environment

**Learn how to set up your environment and verify connectivity with essential research tools.**  
- **1.0 Setup the Environment:**  
  Install and configure all required packages and dependencies for Agno and Contextual AI.
- **1.1 Test Tooling Connectivity:**  
  Ensure that external tools are accessible and working, including:
  - *Tavily (Web Search):* For real-time web information retrieval.
  - *Contextual AI (RAG Agent):* For retrieval-augmented generation and document-based answers.

---

## 2. Build a Simple Agent with Reasoning

**Create basic research agents powered by leading LLMs.**  
- **2.0 Claude 3.7:**  
  Build a simple agent using Anthropic's Claude 3.7 model for general reasoning tasks.
- **2.1 OpenAI o4-mini:**  
  Construct an agent using OpenAI's o4-mini model for general reasoning tasks.

---

## 3. Build a Research Agent with Reasoning and Tool Use

**Combine LLMs with external tools for advanced, auditable research workflows.**  
- **3.0 Configure Tooling for Agents:**  
  Set up and register external tools for agent use.
- **3.1 Research Agents with Tools:**  
  Explore advanced agents that leverage both LLMs and tools:
  - *Claude 3.7 with Web Search:* Integrate web search for up-to-date information.
  - *Claude 3.7 with RAG Search:* Use retrieval-augmented generation for document-grounded answers.
  - *OpenAI o4-mini:* Combine OpenAI's model with tool-augmented reasoning.

---

## Key Features Demonstrated

- **Tool-Augmented Reasoning:**  
  Empower agents to use both language models and external tools (like ContextualAI RAG search and web search)

- **Modular Agent Construction:**  
  Build agents step-by-step, starting from simple reasoning agents to advanced research agents with integrated tool use

- **Step-by-Step Debugging:**  
  Access detailed debug outputs, including intermediate reasoning steps, tool calls, and results, to better understand and audit the agent's decision-making process.

---

You can run this notebook entirely in Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rajshah4/LLM-Evaluation/blob/main/ResearchAgent_Agno_LangFuse.ipynb)

## 1.0 Setup the Environment

### Installing Required Packages

We will install the latest versions of all required libraries.  
- `agno`, `anthropic`, and `openai` are for building and running LLM agents.
- `tavily-python` enables web search capabilities.
- `contextual-client` is used for retrieval-augmented generation (RAG) with Contextual AI.

If running in Colab, you may need to restart the runtime after installation for all packages to be properly recognized.

In [ ]:
%pip install agno anthropic openai tavily-python contextual-client

In [ ]:
import os
import json
from textwrap import dedent

import httpx
from agno.agent import Agent
from agno.models.anthropic import Claude
from agno.tools.reasoning import ReasoningTools
from agno.models.openai import OpenAIChat

## 1.1 Setting Up API Keys and Clients

In this cell, we set up API keys for OpenAI, Anthropic, Tavily, and ContextualAI.  
**Important:** Replace the placeholder API keys with your own credentials.

We then instantiate the clients for:
- `ContextualAI` (for RAG/document search) - [Get Key](https://app.contextual.ai/)
- `TavilyClient` (for web search) - [Get Key](https://app.tavily.com/)

In [ ]:
# Set OpenAI API key as environment variable
os.environ["OPENAI_API_KEY"] = "sk-z"
os.environ["ANTHROPIC_API_KEY"] = "sk-"
os.environ["CONTEXTUAL_API_KEY"] = "key-EMNM"
os.environ["TAVILY_API_KEY"] = "tvly-dev-"

from contextual import ContextualAI
client = ContextualAI()

from tavily import TavilyClient
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"] )

## 1.2 Test the tooling connectivity
Let's make sure our tools work before we connect them up to our agents.

#### Tavily (Web Search)

Let's verify that the Tavily web search API is working.  
A successful response should return a summary or relevant information about the query.  
If you get an error, check your API key and internet connection.

In [ ]:
response = tavily_client.search("Who is Leo Messi?")
print(response)

#### Contextual AI (RAG Agent)

Now, let's test the ContextualAI RAG agent by querying for Tesla's R&D expenses.  
A successful response should return a relevant answer based on indexed documents.  
If you get an error, ensure your API key is correct and the agent ID is valid.

In [ ]:
agent_id = 'c25462d9-f564-4a24-b390-d900ae59b67d'

In [ ]:
query_result = client.agents.query.create(
    agent_id=agent_id,
    messages=[{
        # Input your question here
        "content": "tesla R&D expenses",
        "role": "user"
    }]
)
query_result.message.content

# Build a Simple Agent with Reasoning

Let's start with a simple agent using reasoning as a way to make sure our agents/APIs are working properly.

### 2.0 Claude 3.7

Here, we instantiate a basic research agent using Anthropic's Claude 3.7 model. To show the effect of reasoning, we will try it with and without reasoning.
This agent will answer a general knowledge question and display its reasoning process.

In [ ]:
query = """I own a boutique hotel chain with locations in Lisbon, Berlin, and London. I plan to expand into a new country in Europe, and a city in Asia, in 2026. What factors—such as tourism growth, seasonal occupancy patterns, and local economic indicators—best predict success? I want to be ahead of the curve and be in the next hot city before others. Pull regional travel data, economic statistics, and hotel occupancy rates; analyze trends visually and recommend ideal expansion locations."""

In [ ]:
cl_agent = Agent(
    model=Claude(id="claude-3-7-sonnet-latest"),
    markdown=True,
    reasoning=False,
)

cl_agent.print_response(query,stream=True,show_full_reasoning=True,stream_intermediate_steps=True,)

In [ ]:
cl_agent_r = Agent(
    model=Claude(id="claude-3-7-sonnet-latest"),
    markdown=True,
    reasoning=True,
)

#Easier to process the outputs this way
cl_agent_r.print_response(query,stream=True,show_full_reasoning=True)

### 2.1 OpenAI o4-mini

Now, we create a similar agent using OpenAI's o4-mini model.  
This demonstrates how you can easily swap out LLMs in the Agno framework.

In [ ]:
agent_o4 = Agent(
    model=OpenAIChat(id="o4-mini"),
    markdown=True,
    reasoning=True,
)

agent_o4.print_response(query,stream=True,show_full_reasoning=True,stream_intermediate_steps=True)

# Build a Research Agent with Reasoning and Tool Use

Let's get an agent working that can answer questions by searching the web or retrieving information via a RAG query.

## 3.0 Configure the Tooling for Agents

Next, we'll setup the tools to work with our agentic frameworks

### Defining a ContextualAI RAG Search Tool

This function allows the agent to query the ContextualAI RAG agent for answers grounded in indexed documents (e.g., SEC filings, financial reports).

In [ ]:
def get_contextual_results(query: str) -> str:
    """Search financial 10k and 10q reports for technology companies, such as NVIDIA, and return the results.
    Args: query (str): The query/question to ask the ContextualAI RAG agent.
    Returns: str: The response content from the RAG agent.
    """
    query_result = client.agents.query.create(
        agent_id=agent_id,
        messages=[{
            "content": query,
            "role": "user"
        }]
    )
    return query_result.message.content

In [ ]:
get_contextual_results ("What are Tesla's R&D expenses")

### Defining a Tavily Web Search Tool

This function enables the agent to perform real-time web searches using Tavily, returning up-to-date information from the internet.

In [ ]:
def get_search_tavily(query: str) -> str:
    """
    Search for recent information using Tavily's search API.
    Args: query (str): The search query
    Returns: Dict containing the query and search results or error message
    """
    search_result = tavily_client.search(
        query=query,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=False,
        max_results=5
    )
    return json.dumps(search_result, indent=2)

In [ ]:
get_search_tavily("What is the weather in Chicago")

## 3.1 Research Agents with Tools

We now combine LLMs with both RAG and web search tools.  
This agent can answer questions by searching the web or retrieving information from indexed documents, providing more accurate and up-to-date answers.

#### Claude 3.7 with web search

Let's test the agent with a real-world question.  
The output will show the agent's reasoning steps and how it uses external tools to arrive at an answer.

In [ ]:
reasoning_agent = Agent(
    model=Claude(id="claude-3-7-sonnet-latest"),
    instructions=dedent("""
        You are a research analyst, I want you use your tools to answer the query,
        you should not use your own knowledge, but only use the tools to answer the question.
    """),
    tools=[
        ReasoningTools(add_instructions=True),
        get_contextual_results,
        get_search_tavily
    ],
    show_tool_calls=True,debug_mode=False,markdown=True
)

In [ ]:
reasoning_agent.print_response("What should I wear tomorrow?", stream=True)

#### Claude 3.7 with RAG search

In [ ]:
reasoning_agent.print_response("Analyze the financial effectiveness of NVIDA to Tesla in 2023, use only 10k and 10q", stream=True)

#### OpenAI o4-mini

In [ ]:
reasoning_agent = Agent(
    model=OpenAIChat(id="o4-mini", reasoning_effort="high"),
    instructions=dedent("""
        You are a research analyst, I want you use your tools to answer the query,
        you should not use your own knowledge, but only use the tools to answer the question.
    """),
    tools=[
        ReasoningTools(add_instructions=True),
        get_contextual_results,
        get_search_tavily
    ],
    show_tool_calls=True,debug_mode=False,markdown=True
)

In [ ]:
reasoning_agent.print_response("Analyze the financial effectiveness of NVIDA to Tesla in 2023, use only 10k and 10q", stream=True)

# (Optional) Connect the Research Agent to Langfuse

[Langfuse](https://langfuse.com/) is an experiment management and observability platform for LLM applications.  
Connecting your agent to Langfuse allows you to track, debug, and analyze agent behavior and performance over time.

**Note:** This section is optional and intended for users who want to add observability and experiment tracking to their agent workflows.

Agno has added [code snippets](https://github.com/agno-agi/agno/tree/main/cookbook/observability) for connecting to experiment managent / tracking platforms like [Langfuse](https://langfuse.com/).

You will want to move this code to the start of the notebook to get logging working properly.


In [ ]:
%pip install langfuse opentelemetry-sdk opentelemetry-exporter-otlp

In [ ]:
import base64
import os

from openinference.instrumentation.agno import AgnoInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

LANGFUSE_PUBLIC_KEY = "pk-lf-"
LANGFUSE_SECRET_KEY = "sk-lf-"
LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    "https://us.cloud.langfuse.com/api/public/otel"  # 🇺🇸 US data region
)
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"]="https://cloud.langfuse.com/api/public/otel" # 🇪🇺 EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"]="http://localhost:3000/api/public/otel" # 🏠 Local deployment (>= v3.22.0)

os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"


tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# Start instrumenting agno
AgnoInstrumentor().instrument()

In [ ]:
## Test Query for Langfuse
agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    markdown=True,
    debug_mode=True,
)

agent.print_response("What is currently trending on Twitter?",stream=True)